In [1]:
# math and robotics
import pathlib
import numpy as np
import spatialmath as sm
import quaternion
from comau_msgs.srv import SetIO, SetIORequest

# ROS
import actionlib
import rospy
import hrr_common
from hr_recycler_msgs.msg import (
    AdaptiveGraspingAction, AdaptiveGraspingGoal,
    CuttingAction, CuttingGoal,
    ChangeToolAction, ChangeToolGoal,    
    FingerGraspAction, FingerGraspGoal,    
    PushOpenAction, PushOpenGoal,
    VacuumGraspingAction, VacuumGraspingGoal,
    UnscrewAction, UnscrewGoal
)
from hrr_msgs.msg import CalibrateCobotAction, CalibrateCobotGoal

# set printing and plotting options
np.set_printoptions(precision=5, suppress=True)

from hr_recycler_msgs.msg import CuttingAction, CuttingFeedback, CuttingResult, CuttingGoal, SkillResult, CobotState, MaterialType
import actionlib
from hr_recycler_msgs.msg import ToolType

from hr_recycler_msgs.msg import PlannerAction , PlannerGoal
import hrr_common as hrr_cm

In [2]:
tool_change_pose = [-0.84518,  0.55554, -1.57835,  0.20987, -0.43471, -2.6285 ] #[-0.9376716018,  0.7174308896, -1.2232894897, -0.0096366024, -0.227122277 , -2.4522731304]

In [3]:
rospy.init_node('pcTower_demo')

In [4]:
cobot_ns = hrr_common.ros_utils.fix_prefix(hrr_common.ros_utils.get_param("/cobot_ns"))
change_tool_action_srv_name = hrr_common.ros_utils.get_param(f"{cobot_ns}change_tool_action_srv_name")
calibration_action_srv_name = hrr_common.ros_utils.get_param(f"{cobot_ns}calibration_action_srv_name")
cutting_action_srv_name = hrr_common.ros_utils.get_param(f"{cobot_ns}cutting_action_srv_name")
grinding_action_srv_name = hrr_common.ros_utils.get_param(f"{cobot_ns}grinding_action_srv_name")
pc_opening_action_srv_name = hrr_common.ros_utils.get_param(f"{cobot_ns}pc_opening_action_srv_name")
sensitive_grasping_action_srv_name = hrr_common.ros_utils.get_param(f"{cobot_ns}sensitive_grasping_action_srv_name")
unscrew_action_srv_name = hrr_common.ros_utils.get_param(f"{cobot_ns}unscrew_action_srv_name")
vacuum_pick_place_action_srv_name = hrr_common.ros_utils.get_param(f"{cobot_ns}vacuum_pick_place_action_srv_name")
finger_grasping_action_srv_name= hrr_common.ros_utils.get_param(f"{cobot_ns}finger_grasping_action_srv_name")
#pgrip_client = get_client(finger_grasping_action_srv_name, FingerGraspAction)

def get_client(action_topic, ActionClass, timeout_secs=2.0):
    client = actionlib.SimpleActionClient(action_topic, ActionClass)
    client.wait_for_server(timeout=rospy.Duration(timeout_secs))
    return client
def get_result(cl):
    cl.wait_for_result()
    return cl.get_result()

def cancel(cl):
    cl.cancel_all_goals()
    
def set_screwing_goal(B_normal, B_screw_guess):
    sc_goal = UnscrewGoal()
    sc_goal.screw_location.header.frame_id = "hrr_cobot.base_link"
    sc_goal.surface_normal.header.frame_id = "hrr_cobot.base_link"
    sc_goal.surface_normal.vector = hrr_common.ros_utils.np2vec3(B_normal)
    sc_goal.screw_location.point = hrr_common.ros_utils.np2vec3(B_screw_guess)
    sc_goal.contact_force = 5
    sc_goal.insertion_force = 10
    sc_goal.timeout = 1200.0
    return sc_goal
    
    
vacuum_client = get_client(vacuum_pick_place_action_srv_name, VacuumGraspingAction)
pgrip_client = get_client(finger_grasping_action_srv_name, FingerGraspAction)
tool_change_client = get_client(change_tool_action_srv_name, ChangeToolAction)
shaftgrinder_client=get_client(cutting_action_srv_name, CuttingAction)
grasping_client = get_client(sensitive_grasping_action_srv_name, AdaptiveGraspingAction)
unscrewing_client = get_client(unscrew_action_srv_name, UnscrewAction)
calibrate_client = get_client(calibration_action_srv_name, CalibrateCobotAction)

### Load robot

In [5]:
import hrr_cobot_robot as hrr_rob
cobot = hrr_rob.HrrCobotControl.from_ros("/hrr_cobot")
#cobot.change_tool("nothing")

[rosout] [/pcTower_demo] Initialize MATLAB-Motion planner @ /home/hrr_cobot/_ros/hr_recycler_ws/src/planner/src
[rosout] [/pcTower_demo] connected to Matlab
Shutting down global node /matlab_global_node_23528 with NodeURI http://192.168.2.87:42833/
The value of the ROS_IP environment variable, 192.168.2.87, will be used to set the advertised address for the ROS node.
Initializing global node /matlab_global_node_34130 with NodeURI http://192.168.2.87:39677/
[rosout] [/pcTower_demo] MATLAB-Initialization done


[rosout] vacuum gripper contains two tips. This contradicts default pipeline atm. Use 1
pybullet build time: May  8 2021 05:48:13


[rosout] some functions are not yet compiled. Expect delays upon first call


In [ ]:
#!rosparam get /hrr_cobot/tool_name

# PC Tower disassembly

## Tool Change to Screwdriver

In [ ]:
# cobot.change_tool("nothing")

In [ ]:
# cobot.move_to_joint_pose(tool_change_pose, stochastic=True)
# rospy.sleep(3)
# cobot.open_tool_changer()

tc_goal = ChangeToolGoal()
tc_goal.new_tool.type = tc_goal.new_tool.SCREW_DRIVER
tool_change_client.send_goal(tc_goal)
cancel(tool_change_client)

In [ ]:
# cobot.close_tool_changer()


In [23]:
cobot.change_tool("screwdriver")

True

In [ ]:
#Caliberate
goal = CalibrateCobotGoal()
data_path = pathlib.Path.home() / "Documents" / "skill_recordings"
data_path.mkdir(parents=True, exist_ok=True)
tool_name = hrr_common.ros_utils.get_param(f'{cobot_ns}tool_name')
if tool_name == '':
    tool_name = 'nothing'
goal.calibration_file = str(data_path / f"{tool_name}.pkl")
goal.data_file = str(data_path / f"{tool_name}_calibration.npy")
goal.recalibrate = False
goal.scale_q4 = 0.5
goal.scale_q5 = 0.5
goal.scale_q6 = 0.5

In [ ]:
calibrate_client.send_goal(goal)

# Unscrew screw (1st- back left)

In [8]:
B_normal = sm.base.rotx(0, "deg") @ sm.base.unitvec(np.r_[0., 0.0, 1.0])
B_screw_guess = np.r_[0.687850056696+0.006,  -0.00993826496201+0.028,  0.430110349768]
sc_goal=set_screwing_goal(B_normal=B_normal, B_screw_guess=B_screw_guess)

In [9]:
unscrewing_client.send_goal(sc_goal)
# cancel(unscrewing_client)

In [35]:
cancel(unscrewing_client)

In [13]:
cobot.q

array([ 0.72819,  0.10041, -1.51531, -1.51356,  1.38538, -3.04778])

## Unscrew (2nd-right)

In [10]:
B_normal = sm.base.rotx(0, "deg") @ sm.base.unitvec(np.r_[0., 0.0, 1.0])
B_screw_guess = np.r_[0.749674510877+0.006,  -0.012553101963+0.028,  0.430315881079]
sc_goal=set_screwing_goal(B_normal=B_normal, B_screw_guess=B_screw_guess)

In [11]:
unscrewing_client.send_goal(sc_goal)


In [16]:
#cancel(unscrewing_client)

## Unscrew (2nd-right)

In [12]:
B_normal = sm.base.rotx(0, "deg") @ sm.base.unitvec(np.r_[0., 0.0, 1.0])
B_screw_guess = np.r_[0.689671691919+0.006,  0.0722290824381+0.028,   0.428154349806]
sc_goal=set_screwing_goal(B_normal=B_normal, B_screw_guess=B_screw_guess)

In [16]:
unscrewing_client.send_goal(sc_goal)


In [17]:
cancel(unscrewing_client)

## Unscrew (2nd-right)

In [ ]:
x: 0.750863339196
    y: 0.0701303721073
    z: 0.427483881232

In [18]:
B_normal = sm.base.rotx(0, "deg") @ sm.base.unitvec(np.r_[0., 0.0, 1.0])
B_screw_guess = np.r_[0.750863339196+0.006,  0.0701303721073+0.028,   0.427483881232]
sc_goal=set_screwing_goal(B_normal=B_normal, B_screw_guess=B_screw_guess)

In [19]:
unscrewing_client.send_goal(sc_goal)


In [20]:
cancel(unscrewing_client)

### remember to change collision models

## Change tool to vacuum gripper

In [ ]:
# cobot.change_tool("nothing")

In [7]:
cobot.close_tool_changer()

In [ ]:
# cobot.move_to_joint_pose(tool_change_pose, stochastic=True)
# rospy.sleep(3)
# cobot.open_tool_changer()

tc_goal = ChangeToolGoal()
tc_goal.new_tool.type = tc_goal.new_tool.WSG_50
tool_change_client.send_goal(tc_goal)
tool_change_client.cancel_all_goals()

In [ ]:
# srv = rospy.ServiceProxy("/hrr_cobot/set_digital_io", SetIO)
# srv(SetIORequest(pin=14, state=False))
# srv(SetIORequest(pin=13, state=False))
# srv(SetIORequest(pin=12, state=False))

In [ ]:
# cobot.close_tool_changer()

In [21]:
cobot.change_tool("vacuum")

True

In [11]:
calib_pose = [ 1.34109,  0.09328, -1.44435, -0.01627,  0.67686, -0.00009] # caliberation pose for ecoreset pilot

In [12]:
cobot.move_to_joint_pose(calib_pose, stochastic = True)

> In gpmp2.test3Ddataset>add_obstacle (line 245)
In gpmp2.test3Ddataset (line 47)
In HrRecyclerPerecptioPipeline (line 51)

> In gpmp2.test3Ddataset>add_obstacle (line 245)
In gpmp2.test3Ddataset (line 47)
In HrRecyclerPerecptioPipeline (line 51)

> In gpmp2.test3Ddataset>add_obstacle (line 245)
In gpmp2.test3Ddataset (line 48)
In HrRecyclerPerecptioPipeline (line 51)

> In gpmp2.test3Ddataset>add_obstacle (line 245)
In gpmp2.test3Ddataset (line 48)
In HrRecyclerPerecptioPipeline (line 51)

> In gpmp2.test3Ddataset>add_obstacle (line 245)
In gpmp2.test3Ddataset (line 49)
In HrRecyclerPerecptioPipeline (line 51)

> In gpmp2.test3Ddataset>add_obstacle (line 245)
In gpmp2.test3Ddataset (line 49)
In HrRecyclerPerecptioPipeline (line 51)

> In gpmp2.test3Ddataset>add_obstacle (line 245)
In gpmp2.test3Ddataset (line 50)
In HrRecyclerPerecptioPipeline (line 51)

> In gpmp2.test3Ddataset>add_obstacle (line 245)
In gpmp2.test3Ddataset (line 50)
In HrRecyclerPerecptioPipeline (line 51)

calculat

[rosout] [/pcTower_demo] Planner is done and execution is running ...


## Vacuum grasp the  cover

In [12]:
obj_pos_guess = np.r_[0.654525457641, -0.00305902820621, 0.205666155003] 
obj_orient_guess = np.quaternion(-0.5, 0.5, 0.5, 0.5)

release_pos = np.r_[0.0192,0.5456 , 0.186] 
release_orientation = obj_orient_guess

In [13]:
vac_goal = VacuumGraspingGoal()
vac_goal.object_center.header.frame_id = "hrr_cobot.base_link"
vac_goal.object_center.pose.position = hrr_common.ros_utils.np2vec3(obj_pos_guess)
vac_goal.object_center.pose.orientation = hrr_common.ros_utils.np2quat(obj_orient_guess)

vac_goal.release_pose.header.frame_id = "hrr_cobot.base_link"
vac_goal.release_pose.pose.position = hrr_common.ros_utils.np2vec3(release_pos)
vac_goal.release_pose.pose.orientation = hrr_common.ros_utils.np2quat(release_orientation)

# vac_goal.timeout = 120.0
# vac_goal

In [14]:
vacuum_client.send_goal(vac_goal)


In [10]:
cancel(vacuum_client)

In [13]:
sm.base.r2q(cobot.T_B_E_robot.R)

array([ 0.00826,  0.7347 ,  0.67825, -0.01092])

In [21]:
obj_orient_guess = np.quaternion(0.712022653946,0,-0.702133595634, 0.00567046291769)
obj_orient_guess = quaternion.from_rotation_matrix(quaternion.as_rotation_matrix(obj_orient_guess) @ sm.SE3.Rx(np.rad2deg(-45)).R)

In [50]:
obj_orient_guess = quaternion.from_rotation_matrix(quaternion.as_rotation_matrix(obj_orient_guess) @ sm.SE3.Rx(np.rad2deg(90)).R)

In [40]:
cobot.move_to_joint_pose(cobot.q_calib, stochastic = False)

In [48]:
obj_orient_guess

quaternion(-0.421384067785507, 0.5739440104652, 0.420102418409819, 0.56261683081284)

In [54]:
print(cobot)

Hrr-Cobot state:
FT-data:
current force:	[-1.25244  0.63647  1.6875 ] [N]
current torque:	[-0.59662 -1.22406  0.41571] [Nm]
current wrench:	[-1.24687  0.62884  1.64998 -0.59728 -1.22965  0.40426] [N,Nm]
=>in contact:	True
---
Robot-data:
q:		[-87.75658  29.60633 -86.30174  -0.86436  64.58787 -86.92278][°]
q_dot:		[0. 0. 0. 0. 0. 0.][rad/s]
tau:		[0. 0. 0. 0. 0. 0.][Nm]
pos:		[0.02244 0.60069 0.48305][m]
quat:		-0.00699 <[-0.00401  0.99996 -0.00401]>
tool-pos:	[0.02244 0.60069 0.48305][m]
tool-rpy:	[179.53729  -0.79897 179.53781][°]
robot-status:	moving
---
tool set to vacuum
E_p_EC:=[-0.0725  -0.       0.23018]
R_E_C:=[[ 0.  1.  0.]
 [-1.  0.  0.]
 [ 0.  0.  1.]]
---


In [18]:
cobot.q_calib[-1] = np.deg2rad(-90)

In [28]:
cobot.change_tool("vacuum")

[rosout] vacuum gripper contains two tips. This contradicts default pipeline atm. Use 1


True

In [29]:
cobot.T_B_E_robot

  -2.102e-06  1         4.323e-06  0.4359    
   1         2.102e-06  0         0         
  -9.088e-12  4.323e-06 -1         0.705     
   0         0         0         1         


In [32]:
sm.base.r2q(cobot.T_B_C_robot.R)

array([ 0., -0.,  1., -0.])

In [32]:
print(cobot)

Hrr-Cobot state:
FT-data:
current force:	[-10.51392 -10.77173  29.76855] [N]
current torque:	[ 10.05829 -15.61798  -0.32239] [Nm]
current wrench:	[-10.49    -10.79812  29.71446  10.06351 -15.62533  -0.31463] [N,Nm]
=>in contact:	True
---
Robot-data:
q:		[ 14.42949  59.07733 -52.77855  -0.00001  68.14388 -75.57039][°]
q_dot:		[0. 0. 0. 0. 0. 0.][rad/s]
tau:		[0. 0. 0. 0. 0. 0.][Nm]
pos:		[ 0.72075 -0.18545  0.37787][m]
quat:		-0.00000 <[0.70711 0.70711 0.     ]>
tool-pos:	[ 0.72075 -0.18545  0.37787][m]
tool-rpy:	[ -90.00012    0.      -180.     ][°]
robot-status:	moving
---
tool set to vacuum
E_p_EC:=[-0.0725  -0.       0.23018]
R_E_C:=[[ 0.  1.  0.]
 [-1.  0.  0.]
 [ 0.  0.  1.]]
---


## Dispose PSU

In [ ]:
a = sm.SE3()
[[0,        1     ,    0    ,     0.7246  ],  
[  1   ,      0    ,     0    ,    -0.1398 ],   
  [ 0     ,    0      ,  -1    ,     0.1181  ],  
  [ 0,         0     ,    0  ,       1   ]]

In [26]:
cobot.change_tool("vacuum")

True

In [ ]:
x: 0.661373132974
      y: -0.152754625468
      z: 0.122117626765

In [32]:
obj_pos_guess = np.r_[0.661373132974, -0.152754625468+0.08 ,0.122117626765]
obj_orient_guess = np.quaternion(-0.5, 0.5, 0.5, 0.5)#(0.712022653946,0,-0.702133595634, 0.00567046291769)
release_pos = np.r_[0.0192,0.5456 , 0.186] #0.41 0.48
release_orientation = obj_orient_guess
#obj_orient_guess = quaternion.from_rotation_matrix(quaternion.as_rotation_matrix(obj_orient_guess) @ sm.SE3.Rx(np.rad2deg(-45)).R)

In [33]:
vac_goal = VacuumGraspingGoal()
vac_goal.object_center.header.frame_id = "hrr_cobot.base_link"
vac_goal.object_center.pose.position = hrr_common.ros_utils.np2vec3(obj_pos_guess)
vac_goal.object_center.pose.orientation = hrr_common.ros_utils.np2quat(obj_orient_guess)

vac_goal.release_pose.header.frame_id = "hrr_cobot.base_link"
vac_goal.release_pose.pose.position = hrr_common.ros_utils.np2vec3(release_pos)
vac_goal.release_pose.pose.orientation = hrr_common.ros_utils.np2quat(release_orientation)

# vac_goal.timeout = 120.0
# vac_goal

In [34]:
vacuum_client.send_goal(vac_goal)
# cancel(vacuum_client)

In [42]:
cancel(vacuum_client)

In [41]:
cobot.tool_controller.vacuum = False

def get_SE3_from_pose_stamped(msg):
    p = hrr_common.vec32np(msg.pose.position)
    q = hrr_common.quat2np(msg.pose.orientation)
    return hrr_common.posquat2homog(p, q)

object_center = get_SE3_from_pose_stamped(vac_goal.object_center)
release_center = get_SE3_from_pose_stamped(vac_goal.release_pose)
ee2tip = np.r_[0, 0, 0.20018]

_B_surface_normal = object_center.A[:3, 0]
grasp_pose_EE = hrr_common.utils.calc_goal_pose(_B_surface_normal, object_center.t, y_axis=object_center.A[:3, 1])
            #2. adjust to accomodate tool-tip
grasp_pose = grasp_pose_EE @ sm.SE3(-ee2tip)

release_pose_EE = hrr_common.utils.calc_goal_pose(release_center.A[:3,0], release_center.t, y_axis=release_center.A[:3,1])
release_pose = release_pose_EE @ sm.SE3(-ee2tip) 
#return self.cancel(msg="Thou shalt hard-code")

pre_pose = grasp_pose @ sm.SE3([0, 0, -0.15])

below_pose = grasp_pose @ sm.SE3([0, 0, 0.1])

## Dispose CPU cooler

In [ ]:
x: 0.658834806462
      y: -0.0152477494288
      z: 0.114945146814

In [38]:
obj_pos_guess = np.r_[ 0.658834806462-0.015, -0.0152477494288+0.04,  0.114945146814] #[0.66, 0.17, 0.07] is actual vision data for elamp cover
obj_orient_guess = np.quaternion(-0.5, 0.5, 0.5, 0.5)
release_pos = np.r_[0.0192,0.5456 , 0.186] # will drop the object here, careful!
release_orientation = obj_orient_guess


In [39]:
vac_goal = VacuumGraspingGoal()
vac_goal.object_center.header.frame_id = "hrr_cobot.base_link"
vac_goal.object_center.pose.position = hrr_common.ros_utils.np2vec3(obj_pos_guess)
vac_goal.object_center.pose.orientation = hrr_common.ros_utils.np2quat(obj_orient_guess)

vac_goal.release_pose.header.frame_id = "hrr_cobot.base_link"
vac_goal.release_pose.pose.position = hrr_common.ros_utils.np2vec3(release_pos)
vac_goal.release_pose.pose.orientation = hrr_common.ros_utils.np2quat(release_orientation)

# vac_goal.timeout = 120.0
# vac_goal

In [40]:
vacuum_client.send_goal(vac_goal)
# cancel(vacuum_client)

In [55]:
cancel(vacuum_client)

In [46]:
cobot.close_tool_changer()

## Move to tool change position

In [21]:
cobot.move_to_joint_pose(cobot.q_calib, stochastic=False)

In [47]:
cobot.move_to_joint_pose([ 1.34109,  0.09328, -1.44435, -0.01627,  0.67686, -0.00009], stochastic=True)


> In HrRecyclerPerecptioPipeline (line 24)
calculating signed distance field ...
calculating signed distance field done
Elapsed time is 5.720012 seconds.
Elapsed time is 0.014726 seconds.


[rosout] [/pcTower_demo] Planner is done and execution is running ...


In [38]:
cobot.goTo(sm.SE3(0,-0.2,0)@cobot.T_B_E_robot)